# BSSP Mini Data Case Study

The goal of this case-study is to evaluate the effectiveness of the campaign, and optimize the spend.

In [1]:
import pandas as pd
import numpy as np

In [2]:
## Save both sheets of data as CSV files
## - “Main Data” tab saved as sample_data_main.csv
## - “ProgDSP” tab saved as sample_data_progdsp.csv
main_data = pd.read_csv('sample_data_main.csv')
progdsp = pd.read_csv('sample_data_progdsp.csv')

Peek at what we’re working with 

In [3]:
## Appears to be data at the daily + placement/creative level
main_data.head()

,Date,Campaign,Campaign ID,Site (DCM),Package/Roadblock,Placement,Ad,Creative,Activity,Impressions,Clicks,Media Cost,Total Conversions,Click-through Conversions,View-through Conversions
0,3/1/15,Client Campaign 1,8473474,ProgDSP,P3JL0C_ProgDSP_Used Car Display_BR_ALL_ALL_PKG...,P3JL0K_3rd party_300 x 250_ProgDSP_Prospecting...,300x250 Default Web Ad,0104 _Brand Rigorously_300x250_sys_1501091140_ff,(not set),2636,0,0,0,0,0
1,3/1/15,Client Campaign 1,8473474,ProgDSP,P3JL0C_ProgDSP_Used Car Display_BR_ALL_ALL_PKG...,P3JL0K_3rd party_300 x 250_ProgDSP_Prospecting...,P3JL0K_3rd party_300 x 250_ProgDSP_Prospecting...,0104_Model2 National .9%APR plus 1 payment cre...,(not set),131798,40,0,0,0,0
2,3/1/15,Client Campaign 1,8473474,ProgDSP,P3JL0C_ProgDSP_Used Car Display_BR_ALL_ALL_PKG...,P3JL0K_3rd party_300 x 250_ProgDSP_Prospecting...,P3JL0K_3rd party_300 x 250_ProgDSP_Prospecting...,0104_Model2 National .9%APR plus 1 payment cre...,Homepage,0,0,0,1,0,1
3,3/1/15,Client Campaign 1,8473474,ProgDSP,P3JL0C_ProgDSP_Used Car Display_BR_ALL_ALL_PKG...,P3JL0K_3rd party_300 x 250_ProgDSP_Prospecting...,P3JL0K_3rd party_300 x 250_ProgDSP_Prospecting...,0104_Model2 National .9%APR plus 1 payment cre...,Search click,0,0,0,3,2,1
4,3/1/15,Client Campaign 1,8473474,ProgDSP,P3JL0C_ProgDSP_Used Car Display_BR_ALL_ALL_PKG...,P3JL0K_3rd party_300 x 250_ProgDSP_Prospecting...,P3JL0K_3rd party_300 x 250_ProgDSP_Prospecting...,0104_Model2 National .9%APR plus 1 payment cre...,Results page,0,0,0,3,2,1


In [4]:
## Appears to be data from ProgDSP placements, includes their spend (not found in main_data)
progdsp.head()

,Date,Campaign,Placement,Impressions,Spend
0,3/1/15,Client Campaign 1,P3JL0K_3rd party_300 x 250_ProgDSP_Prospecting...,273220,274.09
1,3/1/15,Client Campaign 1,P3JL0L_3rd party_728 x 90_ProgDSP_3rd Party Ta...,20906,61.17
2,3/1/15,Client Campaign 1,P3JL0M_3rd party_160 x 600_ProgDSP_Retargeting...,5952,34.42
3,3/2/15,Client Campaign 1,P3JL0K_3rd party_300 x 250_ProgDSP_Prospecting...,216137,273.96
4,3/2/15,Client Campaign 1,P3JL0L_3rd party_728 x 90_ProgDSP_3rd Party Ta...,21955,60.38


## Task 1. 
### Evaluate performance of campaign

Not sure what the KPI’s were, so I’ll just look at 
CTR/CPC/CPM/CPA/Conversion Rate (CR), and provide a summary

In [5]:
## Create some functions that can be re-used
def get_cpm(cost,impressions):
    try:
        cpm = (float(cost) * 1000)/float(impressions)
        cpm = '{0:,.2f}'.format(cpm) ## reduce to 2 decimal places, add thousands commas
    except ZeroDivisionError:
        cpm = 0
    return cpm

def get_cpc(cost,clicks):
    try:
        cpc = float(clicks)/float(cost)
        cpc = '{0:,.2f}'.format(cpc)
    except ZeroDivisionError:
        cpc = 0
    return cpc

def get_ctr(clicks,impressions):
    try:
        ctr = (float(clicks)/float(impressions)) * 100
        ctr = '{0:,.4f}'.format(ctr)
    except ZeroDivisionError:
        ctr = 0
    return ctr

def get_cpa(cost,conversions):
    try:
        cpa = float(cost)/float(conversions)
        cpa = '{0:,.2f}'.format(cpa)
    except ZeroDivisionError:
        cpa = 0
    return cpa

def get_cr(conversions,impressions):
    try:
        cr = (float(conversions)/float(impressions)) * 100
        cr = '{0:,.4f}'.format(cr)
    except ZeroDivisionError:
        cr = 0
    return cr

## and if we have a dataframe, we can wrap these up for convenience
def get_stats(df,cost=True):
    df = df.agg('sum').join(pd.DataFrame(df.size(), columns=['Placement Count']))
    df['CTR'] = df.apply(lambda row: float(get_ctr(row['Clicks'],row['Impressions'])), axis=1)
    df['View_CR'] = df.apply(lambda row: float(get_cr(row['View-through Conversions'],row['Impressions'])), axis=1)
    df['Click_CR'] = df.apply(lambda row: float(get_cr(row['Click-through Conversions'],row['Impressions'])), axis=1)
    df['Total_CR'] = df.apply(lambda row: float(get_cr(row['Total Conversions'],row['Impressions'])), axis=1)
    if cost:
        df['CPM'] = df.apply(lambda row: float(get_cpm(row['Media Cost'],row['Impressions'])), axis=1)
        df['CPC'] = df.apply(lambda row: float(get_cpc(row['Media Cost'],row['Clicks'])), axis=1)    
        df['CPA'] = df.apply(lambda row: float(get_cpa(row['Media Cost'],row['Total Conversions'])), axis=1)
    return df

## Calculate these metrics for the entire campaign

For the sake of this case study, I’ll use “Media Cost” as a proxy for “Spend”.

Since ProgDSP placements’ costs aren’t in main_data, we can grab them from progdsp

In [6]:
total_clicks = sum(main_data['Clicks'])
total_impressions = sum(main_data['Impressions'])
total_cost = sum(main_data['Media Cost']) + sum(progdsp['Spend'])
total_conversions = sum(main_data['Total Conversions'])

print 'Total Cost: \n${:,.2f}\n'.format(total_cost)
print 'Total Impressions: \n{:,}\n'.format(total_impressions)
print 'Total Clicks: \n{:,}\n'.format(total_clicks)
print 'Total Conversions: \n{:,}\n'.format(total_conversions)


total_cpm = get_cpm(total_cost,total_impressions)
print 'Overall campaign CPM:\n${0}\n'.format(total_cpm)

total_cpc = get_cpc(total_cost,total_clicks)
print 'Overall campaign CPC:\n${0}\n'.format(total_cpc)

total_cpa = get_cpa(total_cost,total_conversions)
print 'Overall campaign CPA:\n${0}\n'.format(total_cpa)

total_ctr = get_ctr(total_clicks,total_impressions)
print 'Overall campaign CTR:\n{0}%\n'.format(total_ctr)

total_cr = get_cr(total_conversions,total_impressions)
print 'Overall campaign CR:\n{0}%'.format(total_cr)

Total Cost: 
$52,209.58

Total Impressions: 
24,813,998

Total Clicks: 
12,871

Total Conversions: 
4,720

Overall campaign CPM:
$2.10

Overall campaign CPC:
$0.25

Overall campaign CPA:
$11.06

Overall campaign CTR:
0.0519%

Overall campaign CR:
0.0190%


## 2. We can break this down to the daily placement level

In [7]:
## Create a copy of main_data to wrangle and add columns to
p_daily = main_data.copy()

## We want to group by columns 'Placement' and 'Date', as well as the other categoricals.
## Then, aggregate (sum) the delivery metrics
group_col = ['Date','Placement','Site (DCM)']
sum_col = ['Impressions','Clicks','Total Conversions','Click-through Conversions'
           ,'View-through Conversions','Media Cost']

## Remove the other columns, then group by placement/date and sum the rest
p_daily = p_daily[group_col + sum_col]
p_daily = p_daily.groupby(group_col, as_index=False).sum()

## order by Date and Placement
p_daily = p_daily.sort_values(['Date','Placement'])

The media costs for ProgDSP placements aren’t in this dataframe. 

We can grab them from the ProgDSP dataframe.

In [8]:
## Create a copy of progdsp that we can adjust (while preserving the original dataframe)
temp = progdsp.copy()

## Since we’re using Spend as a proxy for media cost, rename the column
temp = temp.rename(columns={'Spend':'Media Cost'})

## Remove irrelevant columns
relevant_columns = ['Date','Media Cost','Placement']
temp = temp[relevant_columns]

## Change the ”0” values of ProgDSP costs to np.NaN so we can use df.fillna function
p_daily.loc[p_daily['Site (DCM)'] == 'ProgDSP','Media Cost'] = np.NaN

## Merge the two dataframes to get ProgDSP costs
p_daily = p_daily.merge(temp,on=['Placement','Date'],how='left')

## Replace the NaN values with their costs
p_daily['Media Cost_x'] = p_daily['Media Cost_x'].fillna(p_daily['Media Cost_y'])

## Clean up the columns
p_daily = p_daily.rename(columns={'Media Cost_x':'Media Cost'})
del p_daily['Media Cost_y']
p_daily['Date'] = pd.to_datetime(p_daily['Date'], format='%m/%d/%y')
p_daily['Date'] = p_daily['Date'].apply(lambda date: date.strftime('%m/%d/%y'))

## Sort by date
p_daily = p_daily.sort_values(['Date'])

In [9]:
## Apply our calculation functions over the observations/rows
p_daily['CPM'] = p_daily.apply(lambda row: float(get_cpm(row['Media Cost'],row['Impressions'])), axis=1)
p_daily['CPC'] = p_daily.apply(lambda row: float(get_cpc(row['Media Cost'],row['Clicks'])), axis=1)
p_daily['CTR'] = p_daily.apply(lambda row: float(get_ctr(row['Clicks'],row['Impressions'])), axis=1)
p_daily['CPA'] = p_daily.apply(lambda row: float(get_cpa(row['Media Cost'],row['Total Conversions'])), axis=1)
p_daily['Total_CR'] = p_daily.apply(lambda row: float(get_cr(row['Total Conversions'],row['Impressions'])), axis=1)
p_daily['View_CR'] = p_daily.apply(lambda row: float(get_cr(row['View-through Conversions'],row['Impressions'])), axis=1)
p_daily['Click_CR'] = p_daily.apply(lambda row: float(get_cr(row['Click-through Conversions'],row['Impressions'])), axis=1)

With this data, we can look at some summary statistics.
These can be compared against campaign KPI’s to determine effectiveness

In [10]:
summary = p_daily.describe()

## Format the columns
for col in ['Impressions','Clicks','Total Conversions','Click-through Conversions','View-through Conversions']:
    summary[col] = summary[col].astype(int)
for col in ['Media Cost','CPM','CPC']:
    summary[col] = summary[col].map('${:,.2f}'.format)
summary.drop(summary.index[0],inplace=True)

summary

,Impressions,Clicks,Total Conversions,Click-through Conversions,View-through Conversions,Media Cost,CPM,CPC,CTR,CPA,Total_CR,View_CR,Click_CR
mean,17316,8,3,1,2,$36.72,$3.13,$0.65,0.617469,7.707708,1.952136,1.858832,0.093305
std,24664,15,6,2,5,$94.01,$4.68,$5.40,6.181648,18.686321,19.366433,19.193648,2.644767
min,0,0,0,0,0,$0.00,$0.00,$0.00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2317,1,0,0,0,$3.77,$0.73,$0.04,0.009900,0.000000,0.000000,0.000000,0.000000
50%,8894,4,1,0,0,$17.76,$1.32,$0.21,0.029300,0.090000,0.002600,0.000000,0.000000
75%,20658,9,3,1,2,$42.06,$4.40,$0.41,0.097300,8.690000,0.042900,0.007200,0.010000
max,274362,114,41,20,36,"$1,630.29",$29.50,$100.00,100.000000,377.470000,500.000000,500.000000,100.000000


## Task 2.
### Optimizing Spend

First, we can see how each placement performed through the entire campaign flight length (rather, the range of the dataset).

Then, we can compare performance of individual placement features.

In [11]:
## Create a copy of our placements from the previous step
placements = p_daily.copy()

## Remove irrelevant columns from our copied dataframe
relevant_columns = ['Placement','Impressions','Clicks','Total Conversions','Click-through Conversions'
                   ,'View-through Conversions','Media Cost']
placements = placements[relevant_columns]

## Organize and summarize the dataframe
placements = placements.groupby(['Placement'])
placements = get_stats(placements)

## Make output more readable
pd.options.display.float_format = '{:,.4f}%'.format
organized_columns = ['Placement Count','Media Cost','CPM','CPC','CPA','CTR','View_CR','Click_CR','Total_CR']
placements = placements[organized_columns]
placements = placements.sort_values(['CPA'])
placements = placements.rename(columns={'Placement Count':'Flight Days'})

## Format our CPM/CPC/CPA columns to have dollar signs instead of percent signs
## Create a copy of this dataframe to format, so we can keep using the raw statistics just in case
placements = placements.copy()
for col in ['Media Cost','CPM','CPC','CPA']:
    placements[col] = placements[col].map('${:,.2f}'.format)

placements

,Flight Days,Media Cost,CPM,CPC,CPA,CTR,View_CR,Click_CR,Total_CR
Placement,,,,,,,,,
P3JL0M_3rd party_160 x 600_ProgDSP_Retargeting_BR_ALL_ALL_STNDDIS_N_P_SA_NTL_STND_GEN_CPM_N_N_NRT,61,$369.23,$4.45,$0.19,$1.12,0.0857%,0.3281%,0.0688%,0.3969%
P3Z12V_3rd party_300 x 250_ProgDSP_Retargeting_BR_ALL_ALL_STNDDIS_N_P_SA_NTL_STND_GEN_CPM_N_N_NRT,60,"$1,635.97",$4.93,$0.18,$1.79,0.0864%,0.2151%,0.0605%,0.2756%
P3Z12Q_3rd party_728 x 90_ProgDSP_Retargeting_BR_ALL_ALL_STNDDIS_N_P_SA_NTL_STND_GEN_CPM_N_N_NRT,60,"$3,640.69",$4.50,$0.13,$2.63,0.0568%,0.1354%,0.0357%,0.1711%
P3JL0K_3rd party_300 x 250_ProgDSP_Prospecting_BR_ALL_ALL_STNDDIS_N_P_SA_NTL_STND_GEN_CPM_N_N_NRT,48,"$1,289.43",$1.14,$0.26,$6.55,0.0292%,0.0152%,0.0022%,0.0174%
P3FYYV_3rd party_160 x 600_DEF_160x600 keyword_DR_All_ALL_STNDDIS_B_P_SA_NTL_STND_N_CPM_N_N_NRT,61,$200.17,$3.00,$0.03,$6.90,0.0105%,0.0435%,0.0000%,0.0435%
P3Z114_3rd party_160 x 600_ProgDSP_Prospecting_West_BR_ALL_ALL_STNDDIS_N_P_SA_OTR_STND_GEN_CPM_N_N_NRT,57,$331.27,$0.83,$0.53,$7.70,0.0439%,0.0032%,0.0075%,0.0107%
P3Z115_3rd party_160 x 600_ProgDSP_Prospecting_South_BR_ALL_ALL_STNDDIS_N_P_SA_OTR_STND_GEN_CPM_N_N_NRT,57,$290.95,$0.80,$0.29,$8.08,0.0232%,0.0019%,0.0080%,0.0099%
P3Z118_3rd party_160 x 600_ProgDSP_Prospecting_East_BR_ALL_ALL_STNDDIS_N_P_SA_OTR_STND_GEN_CPM_N_N_NRT,57,$247.20,$1.09,$0.53,$8.83,0.0573%,0.0049%,0.0075%,0.0123%
P3YB42_3rd party_300 x 250_ProgDSP_Prospecting_East_BR_ALL_ALL_STNDDIS_N_P_SA_OTR_STND_GEN_CPM_N_N_NRT,57,$901.98,$1.12,$0.20,$11.27,0.0222%,0.0050%,0.0050%,0.0100%


The Spend + KPI’s of the above data can be compared against delivery goals of the campaign.
They can be used to allocate future placements, given a campaign goal and budget.

Further analysis can be performed on other features of placements, such as creative copy or ad-size.

### Starting with creative copy

In [12]:
## Get a copy dataframe of all the placements and unique creatives.
creative = main_data.copy()

## Because it’s unclear how spend was allocated among ProgDSP placements’ different creative, 
## we will not calculate CPM/CPC.

## Remove irrelevant columns from our copied dataframe
relevant_columns = ['Creative','Impressions','Clicks','Total Conversions','Click-through Conversions'
                   ,'View-through Conversions']
creative = creative[relevant_columns]

## Extract the actual creative copy from the 'Creative' field
creative['Creative'] = creative.apply(lambda row: row['Creative'].split('_')[1], axis=1)

## Organize and summarize the dataframe
creative = creative.groupby(['Creative'])
creative = get_stats(creative,cost=False)

## Make output more readable
pd.options.display.float_format = '{:,.4f}%'.format
organized_columns = ['Placement Count','CTR','View_CR','Click_CR','Total_CR']
creative = creative[organized_columns]
creative = creative.sort_values(['CTR'],ascending=False)

creative

,Placement Count,CTR,View_CR,Click_CR,Total_CR
Creative,,,,,
Brand Rigorously,1272,0.0879%,0.0000%,0.0772%,0.0772%
Model4 National .9%APR plus 1 payment credit,143,0.0822%,0.0443%,0.0068%,0.0510%
Model4 National .9%APR MY12,1535,0.0759%,0.0219%,0.0110%,0.0329%
Model2 National .9%APR plus 1 payment credit,192,0.0652%,0.0546%,0.0189%,0.0735%
Model4 National .9%APR or 2 credits MY12,1065,0.0607%,0.0117%,0.0070%,0.0187%
Model2 .9%APR MY12,260,0.0366%,0.0036%,0.0018%,0.0054%
Model3 .9%APR MY12,107,0.0315%,0.0012%,0.0011%,0.0024%
Model1 .9%APR MY12,111,0.0273%,0.0015%,0.0015%,0.0030%
Model3 .9%APR or 2 credits MY12,119,0.0203%,0.0014%,0.0010%,0.0025%


Because there wasn’t the data to associate costs with creative, CTR can be examined instead of CPA.

### Next, we’ll do the same thing for ad-size

In [13]:
## Get a copy of all unique placements including their costs
ad_size = p_daily.copy()

## Remove irrelevant columns from our copied dataframe
relevant_columns = ['Placement','Impressions','Clicks','Total Conversions','Click-through Conversions'
                   ,'View-through Conversions','Media Cost']
ad_size = ad_size[relevant_columns]

## Extract the Ad-Size from the 'Placement' field
ad_size['Ad-Size'] = ad_size.apply(lambda row: row['Placement'].split('_')[2], axis=1)
del ad_size['Placement']

## Organize and summarize the dataframe
ad_size = ad_size.groupby(['Ad-Size'])
ad_size = get_stats(ad_size)

## Make output more readable
organized_columns = ['Placement Count','Media Cost','CPM','CPC','CPA','CTR','View_CR','Click_CR','Total_CR']
ad_size = ad_size[organized_columns]
ad_size = ad_size.sort_values(['CPA'])

## Format our CPM/CPC/CPA columns to have dollar signs instead of percent signs
## Create a copy of this dataframe to format, so we can keep using the raw statistics just in case
ad_size_copy = ad_size.copy()
ad_size_copy['Media Cost'] = ad_size_copy['Media Cost'].map('${:,.2f}'.format)
ad_size_copy['CPM'] = ad_size_copy['CPM'].map('${:,.2f}'.format)
ad_size_copy['CPC'] = ad_size_copy['CPC'].map('${:,.2f}'.format)
ad_size_copy['CPA'] = ad_size_copy['CPA'].map('${:,.2f}'.format)

ad_size_copy

,Placement Count,Media Cost,CPM,CPC,CPA,CTR,View_CR,Click_CR,Total_CR
Ad-Size,,,,,,,,,
160 x 600,471,"$3,112.24",$1.71,$0.37,$6.01,0.0633%,0.0195%,0.0089%,0.0284%
728 x 90,408,"$19,826.76",$1.42,$0.17,$10.50,0.0242%,0.0095%,0.0040%,0.0135%
300 x 250,493,"$18,933.57",$2.32,$0.26,$10.75,0.0600%,0.0145%,0.0070%,0.0216%
640 x 480,61,"$10,337.00",$12.00,$0.33,$18.69,0.4000%,0.0039%,0.0602%,0.0642%


By determining which features/combinations of features produce the best results, more spend can be allocated to these placements, increasing the KPI’s.

Further analysis can be done by isolating other features, such as site or some of the other fields within the Placement/Creative strings.

Thanks,
Wilbur Chen